<!-- TABS -->
# Build and train classifier

In [ ]:
# <testing: >
import numpy as np
import torch
from sklearn.datasets import make_blobs
from superduperdb import superduper

db = superduper('mongomock://temp')

# Generate 100 data points with 16 features
X, y = make_blobs(n_samples=100, n_features=16, centers=3, random_state=42)


X_torch = torch.from_numpy(X).float()
y_torch = torch.from_numpy(y).long()
from superduperdb.ext.torch import tensor
t = tensor(torch.float32, (16,))
db.add(t)

from superduperdb.backends.mongodb import Collection
from superduperdb import Document
data = [ Document({'X': t(X_torch[i]), 'y': float(y_torch[i])}) for i in range(100)]
db.execute(Collection('clt').insert_many(data))

input_key = 'X'
select = Collection('clt').find()

In [ ]:
# <tab: Scikit-Learn>
from sklearn.linear_model import LogisticRegression
from superduperdb.ext.sklearn.model import SklearnTrainer, Estimator

# Create a Logistic Regression model
model = LogisticRegression()
model = Estimator(
    object=model,
    identifier='my-model',
    trainer=SklearnTrainer(
        key=(input_key, 'y'),
        select=select,
    )
)

In [ ]:
# <tab: Torch>
from torch import nn
from superduperdb.ext.torch.model import TorchModel
from superduperdb.ext.torch.training import TorchTrainer


class SimpleModel(nn.Module):
    def __init__(self, input_size=16, hidden_size=32, num_classes=3):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Loss function
def my_loss(X, y):
    return torch.nn.functional.binary_cross_entropy_with_logits(
        X[:, 0], y.type(torch.float)
    )


# Create a Logistic Regression model
model = SimpleModel()
model = TorchModel(
    identifier='my-model',
    object=model,         
    trainer=TorchTrainer(
        key=(input_key, 'y'),
        identifier='my_trainer',
        objective=my_loss,
        loader_kwargs={'batch_size': 10},
        max_iterations=100,
        validation_interval=10,
        select=select,
    ),
)

The following command adds the model to the system and trains the model in one command.

In [ ]:
db.apply(model)

In [ ]:
# <testing: >
print(db.load('model', 'my-model').predict_one(X_torch[0]))
print(y_torch[0])